In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('onlinefoods.csv')

In [33]:
df = df.drop(columns=['Unnamed: 12'])

In [34]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
numeric = ['Age', 'Family size', 'latitude', 'longitude', 'Pin code']
df_scaled = df.copy()
standardscaler = StandardScaler()
df_scaled[numeric] = standardscaler.fit_transform(df[numeric])

In [35]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score

df_svm = df

categorical = ['Gender', 'Marital Status', 'Occupation', 'Monthly Income',
               'Educational Qualifications', 'Feedback']

label_encoders = {}
for column in categorical:
    le = LabelEncoder()
    df_svm[column] = le.fit_transform(df_svm[column])
    label_encoders[column] = le

X = df_svm.drop(columns=['Output'])
y = LabelEncoder().fit_transform(df_svm['Output'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
                                                     test_size = 0.2,
                                                     random_state = 42)

svm_model = SVC(kernel = 'linear', C = 1.0, random_state = 42)

pipeline = Pipeline([
    ('scaler', scaler),
    ('svm', svm_model)
])

param_grid = {
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'svm__C': [0.01, 0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto'],
    'svm__degree': [2, 3, 4],
}
grid_search = GridSearchCV(estimator = pipeline, param_grid = param_grid, cv = 5, scoring = 'accuracy', n_jobs = -1, verbose = 1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

import pickle
filename = 'modelfiks.sav'
pickle.dump(best_model,open(filename,'wb'))

Fitting 5 folds for each of 120 candidates, totalling 600 fits
